<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/main/models/base_models_STHyMOUNTAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Entrenamiento de Modelos Baseline para Predicción Espaciotemporal de Precipitación Mensual STHyMOUNTAIN

Este notebook implementa modelos baseline para la predicción de precipitaciones usando datos espaciotemporales.

## 🔍 Implementación de Modelos Avanzados y Técnicas de Validación

Además de los modelos tabulares baseline, implementaremos:

1. **Optimización avanzada con Optuna** para los modelos tabulares XGBoost y LightGBM
2. **Validación robusta** mediante:
   - Hold-Out Validation (ya implementada)
   - Cross-Validation (k=5)
   - Bootstrapping (100 muestras)
3. **Modelos de Deep Learning** para capturar patrones espaciales y temporales:
   - Redes CNN para patrones espaciales
   - Redes ConvLSTM para patrones espaciotemporales

El objetivo es proporcionar una evaluación completa de diferentes enfoques de modelado para la predicción de precipitación en regiones montañosas.

In [38]:
# Configuración del entorno (compatible con Colab y local)
import os
import sys
from pathlib import Path

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')   
    # Si estamos en Colab, clonar el repositorio
    !git clone https://github.com/ninja-marduk/ml_precipitation_prediction.git
    %cd ml_precipitation_prediction
    # Instalar dependencias necesarias
    !pip install -r requirements.txt
    !pip install xarray netCDF4 optuna matplotlib seaborn lightgbm xgboost scikit-learn
    BASE_PATH = '/content/drive/MyDrive/ml_precipitation_prediction'
else:
    # Si estamos en local, usar la ruta actual
    if '/models' in os.getcwd():
        BASE_PATH = Path('..')
    else:
        BASE_PATH = Path('.')

print(f"Entorno configurado. Usando ruta base: {BASE_PATH}")

# Si BASE_PATH viene como string, lo convertimos
BASE_PATH = Path(BASE_PATH)

# Ahora puedes concatenar correctamente
model_output_dir = BASE_PATH / 'models' / 'output'
model_output_dir.mkdir(parents=True, exist_ok=True)

print(f"Directorio para salida de modelos creado: {model_output_dir}")


Entorno configurado. Usando ruta base: ..
Directorio para salida de modelos creado: ../models/output


In [39]:
# 1. Importaciones necesarias
import numpy as np
import pandas as pd
import xarray as xr
import optuna
import pickle
import datetime
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Importaciones para barras de progreso y mejora de visualización
from tqdm.notebook import tqdm, trange
from IPython.display import display, HTML, clear_output
import time

# Configurar visualización más atractiva
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
# Importaciones adicionales para Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, save_model, load_model
from tensorflow.keras.layers import (Dense, Dropout, Conv2D, Conv3D, ConvLSTM2D, BatchNormalization, 
                                   MaxPooling2D, Flatten, Input, concatenate, Reshape, TimeDistributed, UpSampling2D)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

print("TensorFlow versión:", tf.__version__)

# Configurar GPU si está disponible
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print(f"GPU disponible: {physical_devices}")
    # Permitir crecimiento de memoria según sea necesario
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    print("No se detectó GPU. Usando CPU.")

In [ ]:
# 2. Cargar el dataset NetCDF
def load_dataset(file_path):
    """Carga un archivo NetCDF y lo convierte a pandas DataFrame"""
    try:
        # Cargar el archivo NetCDF con xarray
        print(f"Intentando cargar el archivo: {file_path}")
        ds = xr.open_dataset(file_path)
        print("Archivo cargado exitosamente con xarray")

        # Mostrar información del dataset cargado
        print("\nInformación del dataset:")
        print(ds.info())
        print("\nVariables disponibles:")
        for var_name in ds.data_vars:
            print(f"- {var_name}: {ds[var_name].shape}")

        # Convertir a DataFrame
        df = ds.to_dataframe().reset_index()
        return df, ds
    except Exception as e:
        print(f"Error al cargar el archivo NetCDF: {e}")
        return None, None

# Ruta al dataset
data_file = BASE_PATH / 'data' / 'output' / 'complete_dataset_with_features.nc'
print(f"Buscando archivo en: {data_file}")

# Cargar el dataset
df, ds_original = load_dataset(data_file)

# Verificar si se cargó correctamente
if df is not None:
    print(f"Dataset cargado con éxito. Dimensiones: {df.shape}")
    print("\nPrimeras filas del DataFrame:")
    display(df.head())
else:
    print("No se pudo cargar el dataset. Verificar la ruta y el formato del archivo.")

In [ ]:
# 3. Preparación de los datos
if df is not None:
    # Identificar la columna objetivo (precipitación)
    target_column = 'total_precipitation'  # Ajustar si tiene otro nombre en tu dataset

    # Ver si existe 'precip_target' o usar 'total_precipitation'
    if 'total_precipitation' in df.columns:
        target_column = 'total_precipitation'

    print(f"Columna objetivo identificada: {target_column}")

    # Separar variables predictoras y variable objetivo
    feature_cols = [col for col in df.columns if col != target_column and not pd.isna(df[col]).all()]

    # Eliminar columnas no numéricas para los modelos (como fechas o coordenadas si no se usan como features)
    non_feature_cols = ['time', 'spatial_ref']
    feature_cols = [col for col in feature_cols if col not in non_feature_cols]

    # Eliminar filas con valores NaN
    print(f"Filas antes de eliminar NaN: {df.shape[0]}")
    df_clean = df.dropna(subset=[target_column] + feature_cols)
    print(f"Filas después de eliminar NaN: {df_clean.shape[0]}")

    # Separar features y target
    X = df_clean[feature_cols]
    y = df_clean[target_column]

    print(f"\nFeatures seleccionadas ({len(feature_cols)}):\n{feature_cols}")
    print(f"\nVariable objetivo: {target_column}")

In [ ]:
# 4. División del conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensiones del conjunto de entrenamiento: {X_train.shape}")
print(f"Dimensiones del conjunto de prueba: {X_test.shape}")

# 5. Estandarización de variables predictoras
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Guardar el scaler para uso futuro
with open(model_output_dir / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("Escalador guardado en models/output/scaler.pkl")

In [44]:
# 6. Funciones de evaluación y entrenamiento
def evaluar_modelo(y_true, y_pred):
    """Evalúa el rendimiento de un modelo usando múltiples métricas"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

def entrenar_y_evaluar_modelo(modelo, nombre, X_train, y_train, X_test, y_test):
    """Entrena un modelo y evalúa su rendimiento con visualización del progreso"""
    # Crear widget para mostrar información del proceso
    display(HTML(f'<div style="background-color:#f0f8ff; padding:10px; border-radius:5px;">' +
                 f'<h3>🔄 Entrenando modelo: {nombre}</h3>' +
                 f'<div id="status_{nombre}">Estado: Iniciando entrenamiento...</div>' +
                 f'</div>'))
    
    # Tiempo de inicio
    start_time = time.time()
    
    # Entrenar el modelo con seguimiento visual según el tipo
    if hasattr(modelo, 'fit_generator') or nombre in ['XGBoost', 'XGBoost_Optuna', 'LightGBM', 'LightGBM_Optuna']:
        # Para modelos que soportan entrenamiento por lotes como XGBoost, LightGBM
        print(f"Entrenando {nombre} con visualización de progreso...")
        if hasattr(modelo, 'n_estimators'):
            n_estimators = modelo.n_estimators
            for i in tqdm(range(n_estimators), desc=f"Entrenando {nombre}"):
                if i == 0:
                    # Primera iteración, ajuste inicial
                    if nombre.startswith('LightGBM'):
                        # LightGBM tiene parámetro verbose
                        temp_modelo = type(modelo)(n_estimators=1, **{k:v for k,v in modelo.get_params().items() 
                                                                 if k != 'n_estimators' and k != 'verbose'}, verbose=-1)
                    else:
                        temp_modelo = type(modelo)(n_estimators=1, **{k:v for k,v in modelo.get_params().items() 
                                                                if k != 'n_estimators'})
                    temp_modelo.fit(X_train, y_train)
                elif i == n_estimators - 1:
                    # Última iteración, ajuste completo
                    modelo.fit(X_train, y_train)
                
                # Actualizar progreso visual
                if i % max(1, n_estimators // 10) == 0:
                    clear_output(wait=True)
                    display(HTML(f'<div style="background-color:#f0f8ff; padding:10px; border-radius:5px;">' +
                                f'<h3>🔄 Entrenando modelo: {nombre}</h3>' +
                                f'<div id="status_{nombre}">Estado: Progreso {i+1}/{n_estimators} estimadores ({((i+1)/n_estimators*100):.1f}%)</div>' +
                                f'</div>'))
                    time.sleep(0.1)  # Pequeña pausa para actualización visual
        else:
            # Si no tiene n_estimators, entrenamiento directo
            modelo.fit(X_train, y_train)
    else:
        # Para modelos estándar como RandomForest
        modelo.fit(X_train, y_train)
    
    # Tiempo de entrenamiento
    training_time = time.time() - start_time
    
    # Visualizar tiempo de entrenamiento
    display(HTML(f'<div style="background-color:#e6ffe6; padding:10px; border-radius:5px;">' +
                f'<h3>✅ Entrenamiento completado: {nombre}</h3>' +
                f'<div>Tiempo de entrenamiento: {training_time:.2f} segundos</div>' +
                f'</div>'))
    
    print(f"Evaluando rendimiento de {nombre}...")
    predicciones = modelo.predict(X_test)
    rmse, mae, r2 = evaluar_modelo(y_test, predicciones)
    
    # Visualizar métricas con estilo
    display(HTML(f'<div style="background-color:#f5f5dc; padding:10px; border-radius:5px; margin-top:10px;">' +
                f'<h3>📊 Métricas para {nombre}</h3>' +
                f'<table style="width:100%; text-align:left;">' +
                f'<tr><th>Métrica</th><th>Valor</th></tr>' +
                f'<tr><td>RMSE</td><td>{rmse:.4f}</td></tr>' +
                f'<tr><td>MAE</td><td>{mae:.4f}</td></tr>' +
                f'<tr><td>R²</td><td>{r2:.4f}</td></tr>' +
                f'</table></div>'))
    
    return modelo, (rmse, mae, r2)

def guardar_modelo(modelo, nombre):
    """Guarda un modelo entrenado en disco"""
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{nombre}_{timestamp}.pkl"
    with open(model_output_dir / filename, 'wb') as f:
        pickle.dump(modelo, f)
    
    # Visualizar confirmación de guardado
    display(HTML(f'<div style="background-color:#e6ffee; padding:10px; border-radius:5px; margin-top:10px;">' +
                f'<h3>💾 Modelo guardado</h3>' +
                f'<div>Modelo <b>{nombre}</b> guardado como: {filename}</div>' +
                f'</div>'))
    return filename

In [ ]:
# 7. Entrenamiento de modelos base sin optimización

# Inicializar diccionarios para almacenar resultados y modelos
resultados_base = {}  # Para almacenar métricas (RMSE, MAE, R2)
modelos_base = {}     # Para almacenar instancias de modelos
modelos_guardados = {} # Para almacenar nombres de archivos guardados

print("\n🔍 Entrenando modelos baseline sin optimización de hiperparámetros...")

# 1. Modelo RandomForest básico
print("\nEntrenando RandomForest base...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model, rf_metrics = entrenar_y_evaluar_modelo(
    rf_model, 'RandomForest', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['RandomForest'] = rf_metrics
modelos_base['RandomForest'] = rf_model
modelo_file = guardar_modelo(rf_model, 'RandomForest')
modelos_guardados['RandomForest'] = modelo_file

# Visualizar importancia de características para RandomForest
plt.figure(figsize=(12, 6))
feat_importances = rf_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='skyblue')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - RandomForest')
plt.tight_layout()
plt.savefig(model_output_dir / 'randomforest_feature_importance.png')
plt.show()

# 2. Modelo XGBoost básico
print("\nEntrenando XGBoost base...")
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model, xgb_metrics = entrenar_y_evaluar_modelo(
    xgb_model, 'XGBoost', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['XGBoost'] = xgb_metrics
modelos_base['XGBoost'] = xgb_model
modelo_file = guardar_modelo(xgb_model, 'XGBoost')
modelos_guardados['XGBoost'] = modelo_file

# Visualizar importancia de características para XGBoost
plt.figure(figsize=(12, 6))
feat_importances = xgb_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='coral')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - XGBoost')
plt.tight_layout()
plt.savefig(model_output_dir / 'xgboost_feature_importance.png')
plt.show()

# 3. Modelo LightGBM básico
print("\nEntrenando LightGBM base...")
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
lgbm_model, lgbm_metrics = entrenar_y_evaluar_modelo(
    lgbm_model, 'LightGBM', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['LightGBM'] = lgbm_metrics
modelos_base['LightGBM'] = lgbm_model
modelo_file = guardar_modelo(lgbm_model, 'LightGBM')
modelos_guardados['LightGBM'] = modelo_file

# Visualizar importancia de características para LightGBM
plt.figure(figsize=(12, 6))
feat_importances = lgbm_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='lightgreen')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - LightGBM')
plt.tight_layout()
plt.savefig(model_output_dir / 'lightgbm_feature_importance.png')
plt.show()

# Comparar resultados de modelos base
print("\n🔍 Comparación de modelos base sin optimización:")
temp_df = pd.DataFrame(resultados_base, index=['RMSE', 'MAE', 'R2']).T
print("\nOrdenados por RMSE (menor es mejor):")
display(temp_df.sort_values('RMSE'))

# Visualizar comparación de RMSE
plt.figure(figsize=(10, 5))
sns.barplot(x=temp_df.index, y=temp_df['RMSE'])
plt.title('Comparación de RMSE - Modelos Base')
plt.ylabel('RMSE (menor es mejor)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(model_output_dir / 'baseline_rmse_comparison.png')
plt.show()

In [ ]:
# Implementación de función para optimización adaptativa de memoria
import os
import gc
import psutil
import warnings
import optuna

def run_memory_efficient_optimization(model_type, X_train, y_train, X_test, y_test):
    """Ejecuta optimización adaptativa de hiperparámetros considerando uso de memoria.
    
    Args:
        model_type (str): Tipo de modelo a optimizar ('RandomForest', 'XGBoost', 'LightGBM')
        X_train, y_train: Datos de entrenamiento
        X_test, y_test: Datos de prueba
        
    Returns:
        tuple: (mejores_parámetros, modelo_optimizado, métricas)
    """
    print(f"\n📊 Iniciando optimización adaptativa para {model_type}...")
    
    # Verificar memoria disponible
    available_memory_gb = psutil.virtual_memory().available / (1024**3)
    print(f"Memoria RAM disponible: {available_memory_gb:.2f} GB")
    
    # Conjunto de referencias de modelos
    model_types = {
        'RandomForest': RandomForestRegressor,
        'XGBoost': XGBRegressor,
        'LightGBM': LGBMRegressor
    }
    
    if model_type not in model_types:
        raise ValueError(f"Tipo de modelo no soportado: {model_type}. Debe ser uno de: {list(model_types.keys())}")
    
    # Ajustar parámetros según memoria disponible
    if available_memory_gb < 2.0:
        print("⚠️ Memoria baja detectada. Ajustando optimización para uso mínimo de memoria.")
        n_trials = 10
        max_estimators = 50
        max_depth = 6
        subsample = 0.5
    elif available_memory_gb < 8.0:
        print("ℹ️ Memoria moderada detectada. Usando configuración balanceada.")
        n_trials = 30
        max_estimators = 300
        max_depth = 12
        subsample = 0.7
    else:
        print("✅ Memoria amplia detectada. Usando configuración completa para optimización.")
        n_trials = 50
        max_estimators = 500
        max_depth = 20
        subsample = 0.9
        
    # Función objetivo para Optuna según tipo de modelo
    def objective(trial):
        # Parámetros comunes
        common_params = {
            'random_state': 42
        }
        
        # Parámetros específicos por tipo de modelo
        if model_type == 'RandomForest':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, max_estimators),
                'max_depth': trial.suggest_int('max_depth', 3, max_depth),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
                'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
                'n_jobs': -1 if available_memory_gb > 4.0 else 1
            }
        elif model_type == 'XGBoost':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, max_estimators),
                'max_depth': trial.suggest_int('max_depth', 3, max_depth),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, subsample),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'gamma': trial.suggest_float('gamma', 0, 5),
                'tree_method': 'hist',  # Más eficiente en memoria
                'use_label_encoder': False,  # Evitar warning
                'n_jobs': -1 if available_memory_gb > 4.0 else 1
            }
        elif model_type == 'LightGBM':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, max_estimators),
                'max_depth': trial.suggest_int('max_depth', 3, max_depth),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, subsample),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
                'boosting_type': 'gbdt',  # Tipo más eficiente
                'n_jobs': -1 if available_memory_gb > 4.0 else 1,
                'verbose': -1
            }
        
        # Combinar parámetros comunes y específicos
        params.update(common_params)
        
        # Asegurar liberación de memoria antes de crear nuevo modelo
        gc.collect()
        
        # Crear y entrenar modelo
        try:
            model_class = model_types[model_type]
            model = model_class(**params)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            return rmse
        except Exception as e:
            print(f"Error al entrenar modelo con parámetros: {params}")
            print(f"Error: {e}")
            # Retornar un valor alto para que Optuna evite estos parámetros
            return float('inf')
    
    # Configuración de estudio Optuna
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    sampler = optuna.samplers.TPESampler(seed=42)
    
    study_name = f"{model_type}_memory_optimized"
    storage_name = "sqlite:///{}.db".format(model_output_dir / study_name)
    
    # Crear y ejecutar estudio
    try:
        study = optuna.create_study(
            study_name=study_name,
            storage=storage_name,
            direction='minimize',
            sampler=sampler,
            pruner=pruner,
            load_if_exists=True
        )
        
        print(f"Ejecutando {n_trials} pruebas para {model_type}...")
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        
        # Obtener mejores parámetros
        best_params = study.best_params
        best_params['random_state'] = 42
        
        # Si es XGBoost, agregar parámetros adicionales
        if model_type == 'XGBoost':
            best_params['tree_method'] = 'hist'
            best_params['use_label_encoder'] = False
        
        # Entrenar modelo final con mejores parámetros
        print(f"\n✅ Entrenando modelo final {model_type} con mejores parámetros...")
        model_class = model_types[model_type]
        best_model = model_class(**best_params)
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            best_model.fit(X_train, y_train)
        
        # Evaluar modelo final
        y_pred = best_model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"\n📊 Métricas de {model_type} optimizado:")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"R²: {r2:.4f}")
        
        # Añadir al diccionario de resultados
        resultados_base[f"{model_type}_Optuna"] = (rmse, mae, r2)
        
        # Guardar modelo
        filename = f"{model_type}_optimized.pkl"
        with open(model_output_dir / filename, 'wb') as f:
            pickle.dump(best_model, f)
        print(f"Modelo guardado como: {filename}")
        
        # Devolver resultados
        return best_params, best_model, (rmse, mae, r2)
        
    except Exception as e:
        print(f"❌ Error durante la optimización: {e}")
        import traceback
        traceback.print_exc()
        return {}, None, (float('inf'), float('inf'), -float('inf'))

## 🧠 Implementación de Modelos de Deep Learning

A continuación implementaremos modelos basados en redes neuronales profundas para capturar patrones espaciales y temporales en los datos de precipitación.

In [ ]:
# Implementación de modelo CNN para predicción espacial

# Necesitamos reformatear los datos para CNN
# Primero verificamos si hay columnas de coordenadas 'x' y 'y' en nuestros datos
print("\n🔍 Preparando datos para modelos CNN...")

# Verificar si tenemos columnas de coordenadas en nuestros datos
coord_cols = [col for col in feature_cols if col in ['x', 'y', 'latitude', 'longitude', 'lat', 'lon']]

if len(coord_cols) >= 2:
    print(f"Columnas de coordenadas encontradas: {coord_cols}")
    
    # Mapeo de nombres de columnas comunes
    lat_names = ['latitude', 'lat', 'y']
    lon_names = ['longitude', 'lon', 'x']
    
    # Identificar columnas de latitud y longitud
    lat_col = next((col for col in coord_cols if col in lat_names), None)
    lon_col = next((col for col in coord_cols if col in lon_names), None)
    
    if lat_col and lon_col:
        print(f"Usando {lat_col} y {lon_col} como coordenadas para CNN")
        
        # Convertir datos a formato espacial para CNN
        def prepare_spatial_data(X_data, y_data, lat_col, lon_col):
            """Prepara datos espaciales para CNN"""
            # Extraer coordenadas únicas en orden
            lats = sorted(X_data[lat_col].unique())
            lons = sorted(X_data[lon_col].unique())
            
            # Crear diccionarios de mapeo para índices
            lat_to_idx = {lat: idx for idx, lat in enumerate(lats)}
            lon_to_idx = {lon: idx for idx, lon in enumerate(lons)}
            
            # Dimensiones de la grilla
            grid_height = len(lats)
            grid_width = len(lons)
            n_features = X_data.shape[1] - 2  # Restar las dos columnas de coordenadas
            
            # Inicializar arrays
            # Las dimensiones son: [muestras, altura, ancho, canales]
            X_grid = np.zeros((len(X_data), grid_height, grid_width, n_features))
            y_grid = np.zeros((len(y_data), grid_height, grid_width, 1))
            
            # Recorrer todos los datos y ubicarlos en la grilla
            non_coord_cols = [col for col in X_data.columns if col != lat_col and col != lon_col]
            
            for idx in range(len(X_data)):
                lat = X_data.iloc[idx][lat_col]
                lon = X_data.iloc[idx][lon_col]
                
                lat_idx = lat_to_idx[lat]
                lon_idx = lon_to_idx[lon]
                
                # Colocar características en la grilla
                for i, col in enumerate(non_coord_cols):
                    X_grid[idx, lat_idx, lon_idx, i] = X_data.iloc[idx][col]
                
                # Colocar valor objetivo
                y_grid[idx, lat_idx, lon_idx, 0] = y_data.iloc[idx]
            
            return X_grid, y_grid
        
        # Convertir datos de entrenamiento a formato espacial
        print("Convirtiendo datos a formato espacial...")
        try:
            # Reconstruir dataframes a partir de arrays escalados
            X_train_df = pd.DataFrame(X_train_scaled, columns=feature_cols)
            X_test_df = pd.DataFrame(X_test_scaled, columns=feature_cols)
            
            # Preparar datos espaciales
            X_train_spatial, y_train_spatial = prepare_spatial_data(X_train_df, y_train, lat_col, lon_col)
            X_test_spatial, y_test_spatial = prepare_spatial_data(X_test_df, y_test, lat_col, lon_col)
            
            print(f"Datos espaciales preparados:")
            print(f"X_train_spatial: {X_train_spatial.shape}")
            print(f"y_train_spatial: {y_train_spatial.shape}")
            print(f"X_test_spatial: {X_test_spatial.shape}")
            print(f"y_test_spatial: {y_test_spatial.shape}")
            
            # Si la conversión fue exitosa, procedemos con el modelo CNN
            # Modelo CNN para predicción de precipitación
            def create_cnn_model(input_shape):
                """Crea un modelo CNN para predicción espacial"""
                # Usar Input como primera capa para evitar el warning
                inputs = Input(shape=input_shape)
                
                # Primera capa convolucional
                x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
                x = BatchNormalization()(x)
                
                # Segunda capa convolucional
                x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
                x = BatchNormalization()(x)
                x = MaxPooling2D(pool_size=(2, 2))(x)
                x = Dropout(0.25)(x)
                
                # Tercera capa convolucional
                x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x)
                x = BatchNormalization()(x)
                x = MaxPooling2D(pool_size=(2, 2))(x)
                x = Dropout(0.25)(x)
                
                # Cuarta capa convolucional
                x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
                x = BatchNormalization()(x)
                
                # Capa de salida
                outputs = Conv2D(1, kernel_size=(1, 1), activation='linear', padding='same')(x)
                
                # Crear modelo usando API funcional
                model = Model(inputs=inputs, outputs=outputs)
                
                # Compilar modelo
                model.compile(
                    loss='mse',
                    optimizer=Adam(learning_rate=0.001),
                    metrics=['mae']
                )
                
                return model
            
            # Crear y entrenar modelo CNN
            print("\n🧠 Creando y entrenando modelo CNN...")
            input_shape = X_train_spatial.shape[1:]  # (altura, ancho, canales)
            cnn_model = create_cnn_model(input_shape)
            
            # Mostrar resumen del modelo
            cnn_model.summary()
            
            # Callbacks para entrenamiento
            callbacks = [
                EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
                ModelCheckpoint(filepath=model_output_dir / 'cnn_model_best.h5',
                              save_best_only=True, monitor='val_loss')
            ]
            
            # Entrenar modelo
            history = cnn_model.fit(
                X_train_spatial, y_train_spatial,
                validation_split=0.2,
                epochs=100,
                batch_size=32,
                callbacks=callbacks,
                verbose=1
            )
            
            # Evaluar modelo
            print("\n📊 Evaluando modelo CNN...")
            cnn_metrics = cnn_model.evaluate(X_test_spatial, y_test_spatial)
            print(f"Loss (MSE): {cnn_metrics[0]:.4f}")
            print(f"MAE: {cnn_metrics[1]:.4f}")
            
            # Predecir con el modelo
            y_pred_cnn = cnn_model.predict(X_test_spatial)
            
            # Aplanar las predicciones para calcular métricas
            y_test_flat = y_test_spatial.flatten()
            y_pred_flat = y_pred_cnn.flatten()
            
            # Filtrar valores donde y_test_flat > 0 (presumiblemente donde hay datos)
            valid_indices = y_test_flat > 0
            y_test_valid = y_test_flat[valid_indices]
            y_pred_valid = y_pred_flat[valid_indices]
            
            # Calcular métricas
            cnn_rmse = np.sqrt(mean_squared_error(y_test_valid, y_pred_valid))
            cnn_mae = mean_absolute_error(y_test_valid, y_pred_valid)
            cnn_r2 = r2_score(y_test_valid, y_pred_valid)
            
            print(f"RMSE: {cnn_rmse:.4f}")
            print(f"MAE: {cnn_mae:.4f}")
            print(f"R²: {cnn_r2:.4f}")
            
            # Guardar modelo
            cnn_model.save(model_output_dir / 'cnn_model_final.h5')
            print("Modelo CNN guardado como 'cnn_model_final.h5'")
            
            # Visualizar la historia del entrenamiento
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('Pérdida del modelo')
            plt.ylabel('Pérdida')
            plt.xlabel('Época')
            plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
            
            plt.subplot(1, 2, 2)
            plt.plot(history.history['mae'])
            plt.plot(history.history['val_mae'])
            plt.title('Error absoluto medio')
            plt.ylabel('MAE')
            plt.xlabel('Época')
            plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
            
            plt.tight_layout()
            plt.savefig(model_output_dir / 'cnn_training_history.png')
            plt.show()
            
            # Añadir resultados a nuestro diccionario de resultados
            resultados_base['CNN'] = (cnn_rmse, cnn_mae, cnn_r2)
            
        except Exception as e:
            print(f"Error preparando datos espaciales para CNN: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("No se pudieron identificar columnas de latitud y longitud.")
else:
    print("No se encontraron suficientes columnas de coordenadas para implementar CNN.")
    print("El modelo CNN requiere al menos columnas de latitud y longitud.")

In [ ]:
# Implementación de modelo ConvLSTM para predicción espaciotemporal
print("\n🔍 Preparando datos para modelo ConvLSTM...")

# Verificar si tenemos el DataFrame disponible
if 'df' not in locals() or df is None:
    print("DataFrame no disponible, intentando recargarlo...")
    try:
        # Recargar el dataset si no está disponible
        data_file = BASE_PATH / 'data' / 'output' / 'complete_dataset_with_features.nc'
        print(f"Recargando archivo desde: {data_file}")
        df, ds_original = load_dataset(data_file)
        
        if df is None:
            print("Error: No se pudo recargar el DataFrame. Verificar la ruta del archivo.")
    except Exception as e:
        print(f"Error al recargar el DataFrame: {e}")
        import traceback
        traceback.print_exc()

# Verificar si tenemos las variables necesarias
if 'feature_cols' not in locals() or 'target_column' not in locals():
    print("Variables necesarias no definidas, intentando redefinirlas...")
    if df is not None:
        # Identificar la columna objetivo (precipitación)
        target_column = 'total_precipitation'  # Ajustar si tiene otro nombre en tu dataset
        
        # Ver si existe 'precip_target' o usar 'total_precipitation'
        if 'total_precipitation' in df.columns:
            target_column = 'total_precipitation'
        
        # Separar variables predictoras y variable objetivo
        feature_cols = [col for col in df.columns if col != target_column and not pd.isna(df[col]).all()]
        
        # Eliminar columnas no numéricas para los modelos
        non_feature_cols = ['time', 'spatial_ref']
        feature_cols = [col for col in feature_cols if col not in non_feature_cols]

# Si el DataFrame está disponible, continuar con la preparación de datos
if df is not None:
    # Para ConvLSTM necesitamos datos con dimensión temporal
    time_cols = [col for col in df.columns if col in ['time', 'date', 'month', 'year', 'day']]

    if len(time_cols) > 0 and len(coord_cols) >= 2:
        print(f"Columnas temporales encontradas: {time_cols}")
        time_col = time_cols[0]
        
        # Función para preparar datos espaciotemporales
        def prepare_spatiotemporal_data(df, feature_cols, target_column, lat_col, lon_col, time_col, 
                                        sequence_length=3):
            """Prepara datos para ConvLSTM con dimensión espaciotemporal"""
            print("Preparando datos espaciotemporales para ConvLSTM...")
            try:
                # Asegurarnos que la columna temporal está ordenada
                # Verificar el tipo de la columna temporal
                time_dtype = df[time_col].dtype
                print(f"Tipo de dato de columna temporal: {time_dtype}")
                
                if pd.api.types.is_datetime64_any_dtype(df[time_col]):
                    # Ya es datetime, ordenamos
                    df_sorted = df.sort_values(by=time_col)
                else:
                    # Intentar convertir a datetime
                    try:
                        df[time_col] = pd.to_datetime(df[time_col])
                        df_sorted = df.sort_values(by=time_col)
                    except Exception as e:
                        print(f"No se pudo convertir columna temporal a datetime: {e}")
                        # Si no podemos convertir, asumimos que ya está ordenado
                        df_sorted = df
                
                # Extraer coordenadas únicas
                lats = sorted(df_sorted[lat_col].unique())
                lons = sorted(df_sorted[lon_col].unique())
                time_steps = sorted(df_sorted[time_col].unique())
                
                print(f"Dimensiones espaciotemporales:")
                print(f"- Latitudes (filas): {len(lats)}")
                print(f"- Longitudes (columnas): {len(lons)}")
                print(f"- Pasos temporales: {len(time_steps)}")
                
                # Crear mapeos para índices
                lat_to_idx = {lat: idx for idx, lat in enumerate(lats)}
                lon_to_idx = {lon: idx for idx, lon in enumerate(lons)}
                time_to_idx = {time: idx for idx, time in enumerate(time_steps)}
                
                # Filtrar columnas feature eliminando coordenadas y tiempo
                feature_cols_filtered = [col for col in feature_cols if col != lat_col and col != lon_col and col != time_col]
                n_features = len(feature_cols_filtered)
                
                # Dimensiones de la grilla espaciotemporal
                grid_height = len(lats)
                grid_width = len(lons)
                n_timesteps = len(time_steps)
                
                print(f"Características a usar: {n_features}")
                
                # Crear un DataFrame indexado para acceso rápido
                df_indexed = df_sorted.set_index([time_col, lat_col, lon_col])
                
                # Crear matrices 3D para cada paso temporal
                # Las dimensiones son: [tiempo, altura, ancho, features]
                X_spatiotemporal = np.zeros((n_timesteps, grid_height, grid_width, n_features))
                y_spatiotemporal = np.zeros((n_timesteps, grid_height, grid_width, 1))
                
                # Llenar matrices con datos disponibles
                for t_idx, t in enumerate(time_steps):
                    for lat_idx, lat in enumerate(lats):
                        for lon_idx, lon in enumerate(lons):
                            try:
                                # Obtener datos para esta coordenada y tiempo
                                data = df_indexed.loc[(t, lat, lon)]
                                
                                # Llenar características
                                for f_idx, feat in enumerate(feature_cols_filtered):
                                    X_spatiotemporal[t_idx, lat_idx, lon_idx, f_idx] = data[feat]
                                
                                # Llenar target
                                y_spatiotemporal[t_idx, lat_idx, lon_idx, 0] = data[target_column]
                            except KeyError:
                                # Este punto espaciotemporal no existe en los datos
                                pass
                
                # Crear secuencias para ConvLSTM
                # Para cada paso temporal t, usaremos t-sequence_length hasta t-1 para predecir t
                n_sequences = n_timesteps - sequence_length
                
                if n_sequences <= 0:
                    print("No hay suficientes pasos temporales para crear secuencias. Ajustando sequence_length.")
                    sequence_length = max(1, n_timesteps // 2)
                    n_sequences = n_timesteps - sequence_length
                    print(f"Nuevo sequence_length: {sequence_length}, n_sequences: {n_sequences}")
                
                # Crear arrays para secuencias
                X_sequences = np.zeros((n_sequences, sequence_length, grid_height, grid_width, n_features))
                y_sequences = np.zeros((n_sequences, grid_height, grid_width, 1))
                
                for i in range(n_sequences):
                    X_sequences[i] = X_spatiotemporal[i:i+sequence_length]
                    y_sequences[i] = y_spatiotemporal[i+sequence_length]
                
                print(f"Secuencias creadas:")
                print(f"X_sequences: {X_sequences.shape}")
                print(f"y_sequences: {y_sequences.shape}")
                
                # Dividir en train/test
                train_size = int(0.8 * n_sequences)
                X_train = X_sequences[:train_size]
                y_train = y_sequences[:train_size]
                X_test = X_sequences[train_size:]
                y_test = y_sequences[train_size:]
                
                return X_train, y_train, X_test, y_test
            except Exception as e:
                print(f"Error preparando datos espaciotemporales: {e}")
                import traceback
                traceback.print_exc()
                return None, None, None, None
        
        # Intentar preparar datos espaciotemporales
        try:
            X_train_convlstm, y_train_convlstm, X_test_convlstm, y_test_convlstm = prepare_spatiotemporal_data(
                df, feature_cols, target_column, lat_col, lon_col, time_col, sequence_length=3
            )
            
            # Si los datos se preparan correctamente, crear y entrenar modelo ConvLSTM
            if X_train_convlstm is not None:
                print("\n🧠 Creando y entrenando modelo ConvLSTM...")
                
                def create_convlstm_model(input_shape):
                    """Crea un modelo ConvLSTM para predicción espaciotemporal"""
                    model = Sequential([
                        # Capa ConvLSTM
                        ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same',
                                  return_sequences=True, activation='tanh',
                                  input_shape=input_shape),
                        BatchNormalization(),
                        
                        # Segunda capa ConvLSTM
                        ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same',
                                   return_sequences=False, activation='tanh'),
                        BatchNormalization(),
                        
                        # Capa convolucional para reducir mapas de características
                        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                        BatchNormalization(),
                        MaxPooling2D(pool_size=(2, 2)),
                        
                        # Capas finales
                        Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same'),
                        UpSampling2D(size=(2, 2)),  # Restaurar dimensión original
                        Conv2D(filters=1, kernel_size=(3, 3), activation='linear', padding='same')
                    ])
                    
                    # Compilar modelo
                    model.compile(
                        loss='mse',
                        optimizer=Adam(learning_rate=0.001),
                        metrics=['mae']
                    )
                    
                    return model
                
                # Crear modelo ConvLSTM
                input_shape = X_train_convlstm.shape[1:]  # (sequence_length, height, width, features)
                convlstm_model = create_convlstm_model(input_shape)
                
                # Mostrar resumen del modelo
                convlstm_model.summary()
                
                # Callbacks para entrenamiento
                callbacks = [
                    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
                    ModelCheckpoint(filepath=model_output_dir / 'convlstm_model_best.h5',
                                  save_best_only=True, monitor='val_loss')
                ]
                
                # Entrenar modelo
                history = convlstm_model.fit(
                    X_train_convlstm, y_train_convlstm,
                    validation_split=0.2,
                    epochs=100,
                    batch_size=16,
                    callbacks=callbacks,
                    verbose=1
                )
                
                # Evaluar modelo
                print("\n📊 Evaluando modelo ConvLSTM...")
                convlstm_metrics = convlstm_model.evaluate(X_test_convlstm, y_test_convlstm)
                print(f"Loss (MSE): {convlstm_metrics[0]:.4f}")
                print(f"MAE: {convlstm_metrics[1]:.4f}")
                
                # Predecir con el modelo
                y_pred_convlstm = convlstm_model.predict(X_test_convlstm)
                
                # Aplanar las predicciones para calcular métricas
                y_test_flat = y_test_convlstm.flatten()
                y_pred_flat = y_pred_convlstm.flatten()
                
                # Filtrar valores donde y_test_flat > 0 (presumiblemente donde hay datos)
                valid_indices = y_test_flat > 0
                y_test_valid = y_test_flat[valid_indices]
                y_pred_valid = y_pred_flat[valid_indices]
                
                # Calcular métricas
                convlstm_rmse = np.sqrt(mean_squared_error(y_test_valid, y_pred_valid))
                convlstm_mae = mean_absolute_error(y_test_valid, y_pred_valid)
                convlstm_r2 = r2_score(y_test_valid, y_pred_valid)
                
                print(f"RMSE: {convlstm_rmse:.4f}")
                print(f"MAE: {convlstm_mae:.4f}")
                print(f"R²: {convlstm_r2:.4f}")
                
                # Guardar modelo
                convlstm_model.save(model_output_dir / 'convlstm_model_final.h5')
                print("Modelo ConvLSTM guardado como 'convlstm_model_final.h5'")
                
                # Visualizar la historia del entrenamiento
                plt.figure(figsize=(12, 5))
                plt.subplot(1, 2, 1)
                plt.plot(history.history['loss'])
                plt.plot(history.history['val_loss'])
                plt.title('Pérdida del modelo ConvLSTM')
                plt.ylabel('Pérdida')
                plt.xlabel('Época')
                plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
                
                plt.subplot(1, 2, 2)
                plt.plot(history.history['mae'])
                plt.plot(history.history['val_mae'])
                plt.title('Error absoluto medio ConvLSTM')
                plt.ylabel('MAE')
                plt.xlabel('Época')
                plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
                
                plt.tight_layout()
                plt.savefig(model_output_dir / 'convlstm_training_history.png')
                plt.show()
                
                # Añadir resultados a nuestro diccionario de comparación
                resultados_base['ConvLSTM'] = (convlstm_rmse, convlstm_mae, convlstm_r2)
            else:
                print("No se pudieron preparar datos para ConvLSTM.")
        except Exception as e:
            print(f"Error al ejecutar preparación de datos para ConvLSTM: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("No se encontraron columnas temporales o espaciales suficientes para implementar ConvLSTM.")
        print("El modelo ConvLSTM requiere al menos una columna temporal y dos columnas espaciales.")

In [ ]:
# Ejecutar optimización adaptativa de memoria RAM para modelos base
print("\n🔍 Ejecutando optimización adaptativa de memoria RAM para Random Forest...")
rf_params, rf_model_opt, rf_metrics_opt = run_memory_efficient_optimization('RandomForest', X_train_scaled, y_train, X_test_scaled, y_test)

print("\n🔍 Ejecutando optimización adaptativa de memoria RAM para XGBoost...")
xgb_params, xgb_model_opt, xgb_metrics_opt = run_memory_efficient_optimization('XGBoost', X_train_scaled, y_train, X_test_scaled, y_test)

print("\n🔍 Ejecutando optimización adaptativa de memoria RAM para LightGBM...")
lgbm_params, lgbm_model_opt, lgbm_metrics_opt = run_memory_efficient_optimization('LightGBM', X_train_scaled, y_train, X_test_scaled, y_test)

# Resumen de parámetros óptimos encontrados
print("\n📊 Mejores parámetros encontrados para cada modelo:")
print(f"\nRandom Forest: {rf_params}")
print(f"\nXGBoost: {xgb_params}")
print(f"\nLightGBM: {lgbm_params}")